In [89]:
# pip install openai chromadb sentence-transformers FlagEmbedding pymupdf4llm tenacity python-dotenv

In [132]:
# pip show openai chromadb sentence-transformers FlagEmbedding pymupdf4llm tenacity python-dotenv

In [2]:
from dotenv import load_dotenv

In [3]:
load_dotenv('env')

True

In [4]:
# from sentence_transformers import SentenceTransformer
from chromadb.utils import embedding_functions
from FlagEmbedding import FlagReranker
import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings
from FlagEmbedding import BGEM3FlagModel
import pymupdf4llm
from tqdm import tqdm

from tenacity import retry, wait_random_exponential, stop_after_attempt
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import os
from typing import List, Dict, Any, Optional
import logging.config
import json



def embedding_function_bge(text_list):
    return model.encode(text_list, return_dense=True)['dense_vecs']



class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        embeddings = embedding_function_bge(input)
        return embeddings




logger = logging.getLogger(__name__)
AsyncClient = AsyncOpenAI()

model = BGEM3FlagModel('BAAI/bge-m3',  use_fp16=True) 
default_ef = MyEmbeddingFunction()
client = chromadb.PersistentClient(path="chromadb_folder")
# chroma_client = chromadb.HttpClient(host='localhost', port=8000)
# collection = client.get_or_create_collection(name="db_v3",embedding_function=default_ef)
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True)



Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [10]:
def embedding_function_bge(text_list):
    return model.encode(text_list, return_dense=True)['dense_vecs']



class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: Documents) -> Embeddings:
        embeddings = embedding_function_bge(input)
        return embeddings


def process_texts(texts, chunk_size=100, overlap=30):
    """Process a list of texts, splitting them into chunks of specified size with overlap, 
    and accumulating shorter texts."""
    accumulated_words = []  # Accumulate words from texts shorter than chunk_size
    final_chunks = []  # Store the final chunks of text

    for text in texts.split():
        accumulated_words.append(text)

        while len(accumulated_words) >= chunk_size:
            # Take the first chunk_size words for the current chunk
            chunk = " ".join(accumulated_words[:chunk_size])
            final_chunks.append(chunk)
            # Remove words from the start of the accumulated_words, considering overlap
            accumulated_words = accumulated_words[chunk_size - overlap:]
    
    # If there are any remaining words, form the last chunk
    if accumulated_words:
        final_chunks.append(" ".join(accumulated_words))
    
    return final_chunks

def get_pdf_collection(pdf_path):
    """
    Process a PDF file and add its chunks to a collection.
    
    Args:
        pdf_path: Path to the PDF file
        collection: The collection object to add documents to
    """
    md_text = pymupdf4llm.to_markdown(pdf_path,show_progress=True)
    all_chunks = process_texts(md_text, chunk_size=500, overlap=50)
    
    global collection
    collection = client.get_or_create_collection(name=pdf_path,embedding_function=default_ef)
    try:
        for idx, chunk in tqdm(enumerate(all_chunks)):
            id_ = str(idx)
            collection.add(
                documents=[chunk],
                ids=[id_]
            )

        return 'success'
    except Exception as e:
        logger.error(f"Error generating response: {str(e)}", exc_info=True)
        return f'Sorry for inconvenience. Please contact support.'

def get_unique_text_indices(text_list):
    unique_texts = {}
    unique_indices = []
    
    for i, text in enumerate(text_list):
        if text not in unique_texts:
            unique_texts[text] = i
            unique_indices.append(i)
    
    return unique_indices


def get_context(query,n_results=5,top_results=1,threshold = 0.5):
    result = collection.query(query_texts = query,n_results=n_results)
    ## reranker
    texts = result['documents'][0]
    unique_indices = get_unique_text_indices(texts)
    pairs = [[query,texts[x]] for x in unique_indices]
    scores = reranker.compute_score(pairs, normalize=True)
    ## colbert
    query_col = model.encode([query],return_colbert_vecs=True)
    docs_col = model.encode(result['documents'][0],return_colbert_vecs=True)
    colber_scores = []
    for vectors in docs_col['colbert_vecs']:
        colber_scores.append(model.colbert_score(query_col['colbert_vecs'][0],vectors).numpy())

    ## combined+score
    all_score = [scores[i]+colber_scores[i] for i in range(len(scores))]

    valid_indices = [unique_indices[idx] for idx,i in enumerate(all_score) if i>threshold]
    data = [result['documents'][0][index] for index in valid_indices][:top_results]

    return data

def get_full_context(query,top=2):
    result = collection.query(query_texts = query,n_results=5)
    texts = result['documents'][0]
    ids = result['ids'][0]
    unique_indices = get_unique_text_indices(texts)
    unique_docs = [texts[x] for x in unique_indices]
    unique_ids = [ids[x] for x in unique_indices]
    ## colbert
    query_col = model.encode([query],return_colbert_vecs=True)
    docs_col = model.encode(unique_docs,return_colbert_vecs=True)
    colber_scores = []
    for vectors in docs_col['colbert_vecs']:
        colber_scores.append(model.colbert_score(query_col['colbert_vecs'][0],vectors).numpy())
    
    ## full_context_colbert
    full_context_scores = []
    full_context_ids = []
    for id in unique_ids:
        pre_id,post_id = str(int(id)-1), str(int(id)+1)
        # print(pre_id,id,post_id)
        full_context_ids.append([pre_id,id,post_id])
        full_context=collection.get(ids=[f'{pre_id}',f'{id}',f'{post_id}'])['documents']
        full_context = ''.join(full_context)
        full_context_colber_vec = model.encode([full_context],return_colbert_vecs=True)
        full_context_colber_score = model.colbert_score(query_col['colbert_vecs'][0],full_context_colber_vec['colbert_vecs'][0]).numpy()
    
        full_context_scores.append(full_context_colber_score)
    
    all_scores = [2*full_context_scores[i]+0.9*colber_scores[i] for i in range(len(colber_scores))]
    sorted_indices = [index for index, _ in sorted(enumerate(all_scores), key=lambda x: x[1], reverse=True)]
    top_context_ids_list = [full_context_ids[index] for index in sorted_indices][:top]
    flattened_list = np.array(top_context_ids_list).flatten().tolist()
    top_ids = list(set(flattened_list))
    top_context = collection.get(ids=top_ids)['documents']

    return top_context, top_ids


async def generate_response(params: Dict[str, Any]) -> Any:
    """Generate response using OpenAI API with error handling and logging."""
    try:
        logger.info(f"Generating response with model: {params.get('model')}")
        response = await AsyncClient.chat.completions.create(**params)
        logger.info("Response generated successfully")
        return response
    except Exception as e:
        logger.error(f"Error generating response: {str(e)}", exc_info=True)
        return f'Sorry for inconvenience. Please contact support.'

@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
async def chat_completion_request(messages: List[Dict], model='gpt-4o-mini') -> Any:
    """Make a chat completion request with retry logic."""
    try:
        params = {
            'messages': messages,
            'max_tokens': 500,
            'model': model,
            'temperature': 0,
            'response_format': {"type": "json_object"}
        }

        response = await generate_response(params)
        return response
    except Exception as e:
        logger.error(f"Chat completion request failed: {str(e)}", exc_info=True)
        raise


async def get_answer(query):
    context = get_context(query,n_results=5,top_results=2,threshold = 0.5)
    
    user_query = f'Based on below CONTEXT {context} ANSWER the query {query}'
    msg = [{"role": "system", "content": system_instruction},{"role": "system", "content": user_query}]
    
    response = await chat_completion_request(msg)
    output = json.loads(response.choices[0].message.content)

    return output


In [9]:
md_text = pymupdf4llm.to_markdown('handbook.pdf', show_progress=False)
md_text[:100]

'# Zania, Inc.\n\n## Zania Employee Handbook\n\n September 07, 2023\n\n\n-----\n\n### TABLE OF CONTENTS\n\n**COR'

In [11]:
status  = get_pdf_collection('handbook.pdf')
status, collection

Processing handbook.pdf...
[                                        ] (0/46[                                        ] ( 1/46[=                                       ] ( 2/4[==                                      ] ( 3/46[===                                     ] ( 4/4[====                                    ] ( 5/46[=====                                   ] ( 6/4[======                                  ] ( 7/4[======                                  ] ( 8/46[=======                                 ] ( 9/4[========                                ] (10/46[=========                               ] (11/4[==========                              ] (12/46[===========                             ] (13/4[============                            ] (14/46[=============                           ] (15/46[=============                           ] (16/4[==============                          ] (17/46[===============                         ] (18/4[================                        ] (19/46[==

48it [00:01, 31.78it/s]


('success', Collection(name=handbook.pdf))

In [13]:
collection.peek()

{'ids': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9'],
 'embeddings': array([[-0.0086441 ,  0.00251389, -0.0378418 , ...,  0.03457642,
         -0.01916504, -0.00683594],
        [-0.03460693, -0.01473999, -0.05102539, ...,  0.02554321,
         -0.04699707,  0.01019287],
        [-0.04232788,  0.01470184,  0.0131073 , ...,  0.01200104,
         -0.0357666 , -0.01543427],
        ...,
        [-0.00891876,  0.00541687, -0.04766846, ..., -0.00418472,
         -0.0350647 , -0.00565338],
        [-0.01815796,  0.0134964 , -0.05105591, ..., -0.0153656 ,
         -0.04806519, -0.00645828],
        [-0.03308105,  0.00017929, -0.0536499 , ..., -0.02671814,
         -0.06567383,  0.04888916]]),
 'documents': ["# Zania, Inc. ## Zania Employee Handbook September 07, 2023 ----- ### TABLE OF CONTENTS **CORE POLICIES** 4 1.0 WELCOME 4 1.1 A Welcome Policy 4 1.2 At-Will Employment 4 2.0 INTRODUCTORY LANGUAGE AND POLICIES 5 2.1 About the Company 5 2.2 Company Facilities 5 2.3 Ethics Code 5 2.4 M

## querying collection

In [17]:
threshold=0.5
query = 'What is the termination policy of the company?'
result = collection.query(query_texts = query,n_results=5)

In [20]:

texts = result['documents'][0]
ids = result['ids'][0]
unique_indices = get_unique_text_indices(texts)
## reranker
# pairs = [[query,texts[x]] for x in unique_indices]
# scores = reranker.compute_score(pairs, normalize=True)
## colbert
query_col = model.encode([query],return_colbert_vecs=True)
docs_col = model.encode(result['documents'][0],return_colbert_vecs=True)
colber_scores = []
for vectors in docs_col['colbert_vecs']:
    colber_scores.append(model.colbert_score(query_col['colbert_vecs'][0],vectors).numpy())

## combined+score
# all_score = [scores[i]+colber_scores[i] for i in range(len(scores))]


valid_indices = [unique_indices[idx] for idx,i in enumerate(all_score) if i>threshold]
valid_ids = [ids[idx] for idx,i in enumerate(all_score) if i>threshold]
# data = [result['documents'][0][index] for index in valid_indices][:top_results]
valid_indices,valid_ids

([0, 1, 2, 3, 4], ['2', '27', '11', '34', '10'])

In [100]:
def get_full_context(query,top=2):
    result = collection.query(query_texts = query,n_results=5)
    texts = result['documents'][0]
    ids = result['ids'][0]
    unique_indices = get_unique_text_indices(texts)
    unique_docs = [texts[x] for x in unique_indices]
    unique_ids = [ids[x] for x in unique_indices]
    ## colbert
    query_col = model.encode([query],return_colbert_vecs=True)
    docs_col = model.encode(unique_docs,return_colbert_vecs=True)
    colber_scores = []
    for vectors in docs_col['colbert_vecs']:
        colber_scores.append(model.colbert_score(query_col['colbert_vecs'][0],vectors).numpy())
    
    ## full_context_colbert
    full_context_scores = []
    full_context_ids = []
    for id in unique_ids:
        pre_id,post_id = str(int(id)-1), str(int(id)+1)
        # print(pre_id,id,post_id)
        full_context_ids.append([pre_id,id,post_id])
        full_context=collection.get(ids=[f'{pre_id}',f'{id}',f'{post_id}'])['documents']
        full_context = ''.join(full_context)
        full_context_colber_vec = model.encode([full_context],return_colbert_vecs=True)
        full_context_colber_score = model.colbert_score(query_col['colbert_vecs'][0],full_context_colber_vec['colbert_vecs'][0]).numpy()
    
        full_context_scores.append(full_context_colber_score)
    
    all_scores = [2*full_context_scores[i]+0.9*colber_scores[i] for i in range(len(colber_scores))]
    sorted_indices = [index for index, _ in sorted(enumerate(all_scores), key=lambda x: x[1], reverse=True)]
    top_context_ids_list = [full_context_ids[index] for index in sorted_indices][:top]
    flattened_list = np.array(top_context_ids_list).flatten().tolist()
    top_ids = list(set(flattened_list))
    top_context = collection.get(ids=top_ids)['documents']

    return top_context, top_ids


In [105]:
query = 'What are the possible ways a person can be terminated from Zania Inc.? For example, does the company outline termination due to performance issues, misconduct, or violation of company policies? Are there provisions for termination during probation periods, layoffs, or organizational restructuring? Additionally, does Zania Inc. have specific clauses for termination due to ethical breaches, conflict of interest, or redundancy in roles? Please include details about notice periods, severance pay, and any appeals process, if applicable.'
context, context_idx = get_full_context(query,top=2)
context_idx

['12', '11', '10', '13', '9']

In [106]:
context

['purchases over $25 must receive prior approval from your Manager. Submit all sales receipts generated by use of the Company credit card [[weekly/monthly]] to your Manager [[or appropriate department]]. Your Company credit card may not be used for personal reasons. Use of the Company credit card is restricted to approved business related expenses. Any unauthorized purchases made with a credit card issued by the Company will be the cardholder\'s responsibility. You must reimburse any such purchase to the Company within [[#]] days. Immediately report lost or stolen Company cards to your Manager. Failure to follow this policy may result in disciplinary action up to and including discharge. ## 5.0 Performance, Discipline, Layoff, and Termination #### 5.1 Criminal Activity/Arrests Zania, Inc. will report all criminal activity in accordance with applicable law. Involvement in criminal activity while employed by the Company, whether on or off Company property, may result in disciplinary acti

In [96]:
top_context_ids_list = [full_context_ids[index] for index in sorted_indices][:2]
flattened_list = np.array(top_context_ids_list).flatten().tolist()

print(list(set(flattened_list)))

['12', '11', '10', '9']


In [90]:
all_scores, colber_scores, full_context_scores

([1.5768324792385102,
  1.618138349056244,
  1.6482644736766816,
  1.3632394909858703,
  1.6631616175174715],
 [array(0.61041826, dtype=float32),
  array(0.57995284, dtype=float32),
  array(0.55680984, dtype=float32),
  array(0.58661795, dtype=float32),
  array(0.57352144, dtype=float32)],
 [array(0.513728, dtype=float32),
  array(0.5480904, dtype=float32),
  array(0.5735678, dtype=float32),
  array(0.41764167, dtype=float32),
  array(0.57349616, dtype=float32)])

In [95]:
set??

Init signature: set(self, /, *args, **kwargs)
Docstring:     
set() -> new empty set object
set(iterable) -> new set object

Build an unordered collection of unique elements.
Type:           type
Subclasses:     ConstrainedSet, AssumptionsContext, DiophantineSolutionSet, LazySet, LazySet, LazySet

In [34]:
# collection.get(ids=['-1','0'])['documents']

In [44]:
full_context_colber_vec['colbert_vecs'][0]

array([ 0.00892883, -0.02397711,  0.07272871, ...,  0.00454554,
        0.00506709, -0.00931361], dtype=float32)

In [49]:
query_col['colbert_vecs'][0]

array([[-0.03894951,  0.00669283, -0.03746605, ...,  0.01024624,
         0.07320714, -0.02387339],
       [-0.03147411,  0.00867384, -0.03654084, ...,  0.01427744,
         0.06727675, -0.02922596],
       [-0.01926587,  0.01374913, -0.02944535, ...,  0.01025634,
         0.05854911, -0.01618299],
       ...,
       [-0.02583918,  0.01816258, -0.06434744, ..., -0.00254768,
         0.06685262, -0.01595265],
       [-0.03827851, -0.00435914, -0.04195703, ...,  0.00584558,
         0.08012628, -0.0276982 ],
       [ 0.02227906,  0.0050599 , -0.03682962, ...,  0.02933966,
         0.02298273,  0.00760923]], dtype=float32)

In [51]:
# collection.get(ids=[f'{pre_id}',f'{id}',f'{post_id}'])['documents']
# full_context

In [59]:
full_context_data = []
full_context_scores = []
for id in valid_ids:
    pre_id,post_id = str(int(id)-1), (int(id)+1)
    print(pre_id,id,post_id)
    full_context=collection.get(ids=[f'{pre_id}',f'{id}',f'{post_id}'])['documents']
    full_context = ''.join(full_context)
    full_context_colber_vec = model.encode([full_context],return_colbert_vecs=True)
    print(model.colbert_score(query_col['colbert_vecs'][0],full_context_colber_vec['colbert_vecs'][0]).numpy())
    full_context_scores.append(model.colbert_score(query_col['colbert_vecs'][0],full_context_colber_vec['colbert_vecs'][0]).numpy())

    

1 2 3
0.513728
26 27 28
0.5480904
10 11 12
0.5735678
33 34 35
0.41764167
9 10 11
0.57349616


In [66]:
import numpy as np
full_context_scores, colber_scores, np.sum((colber_scores,full_context_scores),axis=0)

([array(0.513728, dtype=float32),
  array(0.5480904, dtype=float32),
  array(0.5735678, dtype=float32),
  array(0.41764167, dtype=float32),
  array(0.57349616, dtype=float32)],
 [array(0.61041826, dtype=float32),
  array(0.57995284, dtype=float32),
  array(0.55680984, dtype=float32),
  array(0.58661795, dtype=float32),
  array(0.57352144, dtype=float32)],
 array([1.1241462, 1.1280432, 1.1303777, 1.0042596, 1.1470176],
       dtype=float32))

In [55]:
# collection.get(ids=['9','10','11'])['documents']
# full_context_colber_vec['colbert_vecs'][0]

In [56]:
# result

In [26]:
# all_chunks = process_texts(md_text, chunk_size=500, overlap=50)
# len(all_chunks)

48

In [111]:
client.list_collections()

['pdf_path', 'handbook.pdf']

In [5]:
# all_chunks[:3]

In [19]:

default_ef(['iam there','ur here'])

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[array([ 0.014725,  0.01081 , -0.01949 , ..., -0.02359 , -0.01715 ,
         0.007538], dtype=float16),
 array([-0.0267  ,  0.02896 , -0.02303 , ...,  0.002542, -0.00825 ,
         0.0432  ], dtype=float16)]

In [30]:
## feed collection

from tqdm import tqdm
for idx,chunk in tqdm(enumerate(all_chunks)):
            

    id_ = str(idx)

    collection.add(
        documents=[chunk],
        ids=[id_])

48it [00:01, 31.21it/s]


In [31]:
query = 'What is the name of the company?'
result = collection.query(query_texts = query,n_results=5)


In [59]:
def get_unique_text_indices(text_list):
    unique_texts = {}
    unique_indices = []
    
    for i, text in enumerate(text_list):
        if text not in unique_texts:
            unique_texts[text] = i
            unique_indices.append(i)
    
    return unique_indices

In [61]:
texts = result['documents'][0]
unique_indices = get_unique_text_indices(texts)
pairs = [[query,texts[x]] for x in unique_indices]
scores = reranker.compute_score(pairs, normalize=True)
scores

[0.030271869424597638,
 0.01704531725343583,
 0.027480692809582097,
 0.020804489950607352,
 0.008711384457705508]

In [43]:
query_col = model.encode([query],return_colbert_vecs=True)
docs_col = model.encode(result['documents'][0],return_colbert_vecs=True)


In [55]:
colber_scores = []
for vectors in docs_col['colbert_vecs']:
    colber_scores.append(model.colbert_score(query_col['colbert_vecs'][0],vectors).numpy())
colber_scores    

[array(0.5121414, dtype=float32),
 array(0.50766957, dtype=float32),
 array(0.5114349, dtype=float32),
 array(0.4899619, dtype=float32),
 array(0.49647462, dtype=float32)]

In [62]:
all_score = [score[i]+colber_scores[i] for i in range(len(score))]

valid_indices = [unique_indices[idx] for idx,i in enumerate(all_score) if i>0.50]
valid_indices

[0, 1, 2, 3, 4]

In [63]:
data = [result['documents'][0][index] for index in valid_indices][:1]
data

['Company can respond with medical attention, as appropriate. The Company will not discriminate or retaliate against employees who take preventative cool-down rest periods in accordance with this policy. #### Workplace Smoking Zania, Inc. is concerned about the effect that smoking and secondhand smoke inhalation can have on its employees and clients. Smoking in the office, client areas, and restrooms is prohibited. ## Trade Secrets and Inventions #### Inventions As necessary, employees will receive a separate notification outlining the ownership of any inventions created by them. ----- # Closing Statement Thank you for reading our handbook. We hope it has provided you with an understanding of our mission, history, and structure as well as our current policies and guidelines. We look forward to working with you to create a successful Company and a safe, productive, and pleasant workplace. Shruti Gupta, CEO Zania, Inc. ----- # Acknowledgment of Receipt and Review By signing below, I ackn

In [64]:
def get_context(query,n_results=5,top_results=1,threshold = 0.5):
    result = collection.query(query_texts = query,n_results=n_results)
    ## reranker
    texts = result['documents'][0]
    unique_indices = get_unique_text_indices(texts)
    pairs = [[query,texts[x]] for x in unique_indices]
    scores = reranker.compute_score(pairs, normalize=True)
    ## colbert
    query_col = model.encode([query],return_colbert_vecs=True)
    docs_col = model.encode(result['documents'][0],return_colbert_vecs=True)
    colber_scores = []
    for vectors in docs_col['colbert_vecs']:
        colber_scores.append(model.colbert_score(query_col['colbert_vecs'][0],vectors).numpy())

    ## combined+score
    all_score = [score[i]+colber_scores[i] for i in range(len(score))]

    valid_indices = [unique_indices[idx] for idx,i in enumerate(all_score) if i>threshold]
    data = [result['documents'][0][index] for index in valid_indices][:top_results]

    return data


    

In [113]:
from tenacity import retry, wait_random_exponential, stop_after_attempt
from tqdm import tqdm
from openai import OpenAI, AsyncOpenAI
import os
from typing import List, Dict, Any, Optional
import logging.config
logger = logging.getLogger(__name__)


AsyncClient = AsyncOpenAI()


async def generate_response(params: Dict[str, Any]) -> Any:
    """Generate response using OpenAI API with error handling and logging."""
    try:
        logger.info(f"Generating response with model: {params.get('model')}")
        response = await AsyncClient.chat.completions.create(**params)
        logger.info("Response generated successfully")
        return response
    except Exception as e:
        logger.error(f"Error generating response: {str(e)}", exc_info=True)
        return f'Sorry for inconvenience. Please contact support.'

@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
async def chat_completion_request(messages: List[Dict], model='gpt-4o-mini') -> Any:
    """Make a chat completion request with retry logic."""
    try:
        params = {
            'messages': messages,
            'max_tokens': 500,
            'model': model,
            'temperature': 0,
            'response_format': {"type": "json_object"}
        }

        response = await generate_response(params)
        return response
    except Exception as e:
        logger.error(f"Chat completion request failed: {str(e)}", exc_info=True)
        raise


In [117]:
import json

system_instruction = '''Ideal Output Format
The output should be a structured JSON blob that question with its corresponding answer.
Answers should be word to word match if the question is a word to word match
If the answer is low confidence, reply with “Data Not Available'''


async def get_answer(query):
    # context = get_context(query,n_results=5,top_results=2,threshold = 0.5)
    context, context_idx = get_full_context(query, n_results=5, top=2)
    user_query = f'Based on below CONTEXT {context} ANSWER the query {query}'
    msg = [{"role": "system", "content": system_instruction},{"role": "user", "content": user_query}]
    
    response = await chat_completion_request(msg)
    output = json.loads(response.choices[0].message.content)

    return output

In [120]:
async def chat_completion_request(messages: List[Dict], tools=None, model='gpt-4o-mini', stream=False) -> Any:
    """Make a chat completion request with retry logic."""
    try:
        params = {
            'messages': messages,
            'max_tokens': 1500,
            'model': model,
            'temperature': 0,
            'tools': tools,
            'tool_choice': "auto",
            'stream': stream,
        }
        response = await generate_response(params)
        return response
    except Exception as e:
        logging.error(f"Chat completion request failed: {str(e)}", exc_info=True)
        raise

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_answer",
            "description": "Use this function to get answers based on context documents from database to user questions. The documents are purely based on user db.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "User query in string.",
                    }
                },
                "required": ["query"],
            },
        },
    },
]

system_prompt = '''
Your task is to answer user questions. The user can ask a single question or multiple questions.
You have access to the get_context tool to return answers based on user single/multiple questions.
Return your thoughts if access database, ask user to wait if needed.
'''


In [122]:
system_msg = [{"role": "system", "content": system_prompt}]
user_msg = [{"role": "user", "content": query}]
feed_msg = system_msg + user_msg

chat_answer = await chat_completion_request(feed_msg, tools=tools, stream=False)

In [127]:
chat_answer

ChatCompletion(id='chatcmpl-AnWOX5Hx2akcorGTHZ4Vopa5jdq8m', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uM2x01E7rKvP4JxcM28y9Py3', function=Function(arguments='{"query":"termination policies at Zania Inc."}', name='get_answer'), type='function')]))], created=1736365961, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=21, prompt_tokens=217, total_tokens=238, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [124]:
chat_answer.usage

CompletionUsage(completion_tokens=21, prompt_tokens=217, total_tokens=238, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [129]:
chat_answer.usage.total_tokens

238

In [126]:
chat_answer.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_uM2x01E7rKvP4JxcM28y9Py3', function=Function(arguments='{"query":"termination policies at Zania Inc."}', name='get_answer'), type='function')])

In [128]:
chat_answer.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_uM2x01E7rKvP4JxcM28y9Py3', function=Function(arguments='{"query":"termination policies at Zania Inc."}', name='get_answer'), type='function')]

In [ ]:
response = await chat_completion_request(msg)

In [119]:
for query in questions:
    print('###'*25)
    print(query)
    ans = await get_answer(query)
    print(ans['answer'])
    

NameError: name 'questions' is not defined

In [82]:
get_context(query,n_results=5,top_results=1,threshold = 0.5)

["requests for vacation when possible, taking business needs into consideration. When multiple employees request the same time off, their [[length of employment/seniority/collective-bargaining agreement]] may determine priority in scheduling vacation times. You must take vacation in increments of at least [[# of hours/days]]. _During a Leave of Absence_ The Company may require you to use any unused vacation during disability or family medical leave, or any other leave of absence, where permissible under local, state, and federal law. [[EMPLOYERS WHO USE AN ACCRUAL METHOD MUST ADD THE FOLLOWING LANGUAGE: You will not accrue vacation during unpaid leaves of absence, or other periods of inactive service, unless vacation accrual is required by applicable federal, state, or local law.]] _Carryover_ [[EMPLOYERS MUST CHOOSE ONE:]] [[Unused vacation can be carried over to the following year [You may want to explain conditions for carryover: e.g., on or about your anniversary date/at the beginn

In [5]:
questions = [
    'What is the name of the company?',

'Who is the CEO of the company?',

'What is their vacation policy?',

'What is the termination policy?']

In [81]:
query

'What is their vacation policy?'

In [8]:
for q in questions:
    print(f'"{q}"',end=' ')

"What is the name of the company?" "Who is the CEO of the company?" "What is their vacation policy?" "What is the termination policy?" 

In [112]:
"What are the possible ways a person can be terminated from Zania Inc.? For example, does the company outline termination due to performance issues, misconduct, or violation of company policies? Are there provisions for termination during probation periods, layoffs, or organizational restructuring? Additionally, does Zania Inc. have specific clauses for termination due to ethical breaches, conflict of interest, or redundancy in roles? Please include details about notice periods, severance pay, and any appeals process, if applicable."

'What are the possible ways a person can be terminated from Zania Inc.? For example, does the company outline termination due to performance issues, misconduct, or violation of company policies? Are there provisions for termination during probation periods, layoffs, or organizational restructuring? Additionally, does Zania Inc. have specific clauses for termination due to ethical breaches, conflict of interest, or redundancy in roles? Please include details about notice periods, severance pay, and any appeals process, if applicable.'

In [130]:
client.list_collections()

['pdf_path', 'handbook.pdf']